In [ ]:
import json

import matplotlib.pyplot as plt
import numpy as np
from scipy import stats


In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def logit(y):
    return np.log(y / (1 - y))

def logit_derivative(y):
    return 1 / (y - y**2)

def sample(logit_mean, logit_std, size):
    return sigmoid(np.random.normal(logit_mean, logit_std, size))

def evaluate(y, logit_mean, logit_std, factor=1):
    return stats.norm.pdf(logit(y / factor), logit_mean, logit_std) * logit_derivative(y / factor) / factor 


In [ ]:
json_files = [
    ("Los Angeles", "/data/shared/coanet_results/best_regional_fits/time=2021-06-08T22:40:30,git_hash=19322b5+,noise=0.0005,lr=5.0e-6,lead_in=7,inf_thresh=0.08,duration=163,inf_sat=0.5,iters=40,fixed_E0=0.08,seed=1,county=losangeles-exp,noise_fn=day_scaling,sc=5.0e-5,/posterior_params.json"),
    ("Middlesex", "/data/shared/coanet_results/best_regional_fits/time=2021-06-08T22:40:31,git_hash=19322b5+,noise=0.00025,lr=5.0e-6,lead_in=7,inf_thresh=0.02,duration=163,inf_sat=0.5,iters=40,fixed_E0=0.02,seed=1,county=middlesex-exp,noise_fn=day_scaling,sc=5.0e-5,/posterior_params.json"),
    ("Miami-Dade", "/data/shared/coanet_results/best_regional_fits/time=2021-06-07T22:26:37,git_hash=19322b5+,noise=0.00025,lr=5.0e-6,lead_in=7,inf_thresh=0.02,duration=163,inf_sat=0.5,iters=40,fixed_E0=0.02,seed=1,county=miamidade-exp,noise_fn=day_scaling,sc=5.0e-5,/posterior_params.json"),
]

In [ ]:
def plot_E0(county_name, params):
    # Plot E0s
    e0_logit_means = np.array(params["post_E0_logit_means"])
    e0_logit_std = np.exp(params["post_E0_logit_std_L"])

    print(e0_logit_means.shape)
    print(e0_logit_std.shape)

    nrows = 4
    ncols = 5
    fig, ax = plt.subplots(nrows, ncols, figsize=(10, 5), constrained_layout=True)

    # Compute PDF for each county
    xs = np.linspace(0.000001, 0.01, 1000) 
    result = []
    for idx, county_logit_mean in enumerate(e0_logit_means):
        tmp = []
        for i in xs:
            tmp.append(evaluate(i, county_logit_mean, e0_logit_std, 3))
        result.append(np.array(tmp))

    for idx, data in enumerate(result):
        ax_idx = np.unravel_index(idx, (nrows, ncols))
        ax[ax_idx].plot(xs, data)
        ax[ax_idx].set_xticks([0, 0.005, 0.01])
        ax[ax_idx].set_ylim([0, 300])
        ax[ax_idx].set_title(f"CBG {idx+1}")
        if ax_idx[0] < nrows - 1:
            # Only show xaxis for final row
            ax[ax_idx].xaxis.set_visible(False)
        if ax_idx[1] > 0:
            # Only show yaxis for first col
            ax[ax_idx].yaxis.set_visible(False)

    fig.suptitle(f"Posterior Distribution of ρc by CBG ({county_name})")
    name = f"posterior_params.{county_name.replace(' ', '_')}.E0.png"
    fig.savefig(name, dpi=300, bbox_inches="tight")
    plt.show()

In [ ]:
def plot_betaE(county_name, params):
    # Plot BetaEs
    betaE_logit_means = np.array(params["post_βE_logit_means"])
    betaE_logit_stds = np.exp(params["post_βE_logit_std_Ls"])

    print(betaE_logit_means.shape)
    print(betaE_logit_stds.shape)

    fig, ax = plt.subplots(6, 1, figsize=(10, 10), constrained_layout=True)

    xs = np.linspace(0.0001, 0.6, 1000) 
    result = []
    for idx, (logit_mean, logit_std) in enumerate(zip(betaE_logit_means, betaE_logit_stds)):
        tmp = []
        for i in xs:
            tmp.append(evaluate(i, logit_mean, logit_std))
        result.append(np.array(tmp))

    for idx, data in enumerate(result):
        ax_idx = np.unravel_index(idx, (6,))
        ax[ax_idx].plot(xs, data)
        ax[ax_idx].set_xticks([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
        # ax[ax_idx].set_ylim([0, 300])
        ax[ax_idx].set_title(f"Knot {idx}")
        if ax_idx[0] < 5:
            ax[ax_idx].xaxis.set_visible(False)

    fig.suptitle(f"Posterior Distribution of βE by Knot ({county_name})")
    name = f"posterior_params.{county_name.replace(' ', '_')}.betaE.png"
    fig.savefig(name, dpi=300, bbox_inches="tight")
    plt.show()


In [ ]:
def plot_betaI(county_name, params):
    # Plot BetaEs
    betaI_logit_means = np.array(params["post_βI_logit_means"])
    betaI_logit_stds = np.exp(params["post_βI_logit_std_Ls"])

    print(betaI_logit_means.shape)
    print(betaI_logit_stds.shape)

    fig, ax = plt.subplots(6, 1, figsize=(10, 10), constrained_layout=True)

    xs = np.linspace(0.0001, 0.2, 1000) 
    result = []
    for idx, (logit_mean, logit_std) in enumerate(zip(betaI_logit_means, betaI_logit_stds)):
        tmp = []
        for i in xs:
            tmp.append(evaluate(i, logit_mean, logit_std))
        result.append(np.array(tmp))


    for idx, data in enumerate(result):
        ax_idx = np.unravel_index(idx, (6,))
        ax[ax_idx].plot(xs, data)
        ax[ax_idx].set_xticks([0, 0.1, 0.2])
        # ax[ax_idx].set_ylim([0, 300])
        ax[ax_idx].set_title(f"Knot {idx}")
        if ax_idx[0] < 5:
            ax[ax_idx].xaxis.set_visible(False)

    fig.suptitle(f"Posterior Distribution of βI by Knot ({county_name})")
    name = f"posterior_params.{county_name.replace(' ', '_')}.betaI.png"
    fig.savefig(name, dpi=300, bbox_inches="tight")
    plt.show()


In [ ]:
for county, json_file in json_files:
    with open(json_file, "r") as f:
        params = json.load(f)
    plot_E0(county, params)
    plot_betaE(county, params)
    plot_betaI(county, params)